In [1]:
%matplotlib inline
import os
import re
import io
import itertools
import pprint

from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.plotting import figure
from bokeh.models import Range1d, Legend
from bokeh.palettes import all_palettes

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr

import climatools.lblnew.bestfit_params as bestfits
from climatools.lblnew import setup_bestfit, setup_overlap
import climatools.lblnew.pipeline as pipe_lblnew
import climatools.cliradlw.setup as setup_cliradlw
import climatools.cliradlw.pipeline as pipe_cliradlw

import climatools.html.html as climahtml
from climatools.lblnew.dataio import *
from climatools.plot.plot import *


import IPython.display as display

The history saving thread hit an unexpected error (OperationalError('database is locked',)).History will not be written to the database.


In [2]:
%run param.py

In [3]:
def load_output_file(path_csv):
    '''
    Load lblnew output .csv file to xarray.Dataset
    
    Parameters
    ----------
    path_csv: str
              Path to the .csv file to be loaded.
    ds: xarray.Dataset
        Data in the input file in the form of an xarray.Dataset.
    '''
    toindex = ['band', 'pressure', 'igg', 'g']    
    df = pd.read_csv(path_csv, sep=r'\s+')
    df = df.set_index([i for i in toindex if i in df.columns])
    df = df.rename(columns={'sfu': 'flug',
                            'sfd': 'fldg',
                            'fnet': 'fnetg',
                            'coolr': 'coolrg'})
    ds = xr.Dataset.from_dataframe(df)

    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    return ds

In [4]:
def lblnew_setup(param=None):
    if 'ng_refs' in param:
        return {'setup': setup_bestfit,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wfluxg.dat',
                'fname_cool_wgt': 'output_wcoolrg.dat'}
    else:
        return {'setup': setup_overlap,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wflux.dat',
                'fname_cool_wgt': 'output_wcoolr.dat'}

    
def load_lblnew_data(param):
    
    fname_dsname = [('fname_flux_crd', 'ds_flux_crd'),
                    ('fname_cool_crd', 'ds_cool_crd'),
                    ('fname_flux_wgt', 'ds_flux_wgt'),
                    ('fname_cool_wgt', 'ds_cool_wgt')]
    
    d = lblnew_setup(param)
    dir_fortran = pipe_lblnew.get_dir_case(param, setup=d['setup'])
    
    data_dict = {}
    for fname, dsname in fname_dsname:
        fpath = os.path.join(dir_fortran, d[fname])
        data_dict[dsname] = load_output_file(fpath)
    return data_dict
    

    
    

In [5]:
d = load_lblnew_data(PARAM_LBLNEW)

DS_FLUX_CRD = d['ds_flux_crd']
DS_COOL_CRD = d['ds_cool_crd']
DS_FLUX_WGT = d['ds_flux_wgt']
DS_COOL_WGT = d['ds_cool_wgt']

In [6]:
DIR_FORTRAN = pipe_cliradlw.get_fortran_dir(PARAM, 
                                            setup=setup_cliradlw)

PATH_FLUX = os.path.join(DIR_FORTRAN, 'output_flux.dat')
PATH_COOL = os.path.join(DIR_FORTRAN, 'output_coolr.dat')

DS_FLUX = load_output_file(PATH_FLUX)
DS_COOL = load_output_file(PATH_COOL)

In [7]:
output_notebook()

Loading BokehJS ...

In [8]:
def fmt_cool(ds_in):
    ds = ds_in.copy(deep=True)
    if 'igg' in ds.dims:
        ds = ds.sel(igg=1)

    if 'g' in ds.dims:
        ds = ds.sum('g')
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band')
        except ValueError:
            ds = ds.sum('band')
                
    return ds['coolrg']


def pltdata_cooling_1(ds_cool=None,
                      ds_cool_crd=None, ds_cool_wgt=None):
    


    colors = all_palettes['Set1'][4]
    data = [{'label': 'CLIRAD `coolr_bands`',
             'srs': fmt_cool(ds_cool),
             'line_dash': 'dashed', 'line_width': 5,
             'color': colors[0], 'alpha': .6},
            {'label': 'CRD',
             'srs': fmt_cool(ds_cool_crd),
             'line_dash': 'solid', 'line_width': 1.5,
             'marker': 'circle', 'marker_size': 5,
             'color': colors[2], 'alpha': 1,},
            {'label': 'WGT igg=10 wgt_flux=2',
             'srs': fmt_cool(ds_cool_wgt),
             'line_dash': 'solid', 'line_width': 3,
             'marker': 'square', 'marker_size': 5,
             'color': colors[3], 'alpha': .6}]
    return data


def nice_xlims(pltdata=None, prange=None):
    
    def get_slice(srs):
        return srs.sel(pressure=slice(*prange))
    
    srss = [d['srs'] for d in pltdata]
    vmin = min([get_slice(srs).min() for srs in srss])
    vmax = max([get_slice(srs).max() for srs in srss])
    dv = (vmax - vmin) * .01
    return float(vmin - dv), float(vmax + dv)
    

def plt_cooling_bokeh(pltdata=None):
    '''
    Plot a list of cooling rate profiles using bokeh.
    '''
    ymin = 1e-2 
    ymax = 1020 
        
    p1 = figure(title="Linear pressure scale", 
                plot_width=400)        
    xmin, xmax = nice_xlims(pltdata, prange=(50, 1050))
    for d in pltdata:
        if 'marker' in d:
            getattr(p1, d['marker'])(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=.7)
        p1.line(d['srs'].values, 
                d['srs'].coords['pressure'].values,
                color=d['color'], alpha=d['alpha'], 
                line_width=d['line_width'], line_dash=d['line_dash'])
    
    p1.y_range = Range1d(ymax, ymin)
    p1.yaxis.axis_label = 'pressure [mb]'   
    p1.x_range = Range1d(xmin, xmax)
    p1.xaxis.axis_label = 'cooling rate [K/day]'
            
    p2 = figure(title='Log pressure scale', y_axis_type='log',
                plot_width=560)
    xmin, xmax = nice_xlims(pltdata, prange=(.01, 200))
    
    rs = []
    for d in pltdata:
        rd = []
        if 'marker' in d:
            r_mark = getattr(p2, d['marker'])(d['srs'].values, 
                        d['srs'].coords['pressure'].values,
                        color=d['color'], alpha=.7)
            rd.append(r_mark)
        r_line = p2.line(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=d['alpha'], 
                    line_width=d['line_width'], line_dash=d['line_dash'])
        rd.append(r_line)
      
        rs.append(rd)
        
    p2.y_range = Range1d(ymax, ymin)  
    p2.yaxis.axis_label = 'pressure [mb]'
    
    p2.x_range = Range1d(xmin, xmax)
    p2.xaxis.axis_label = 'cooling rate [K/day]'
    
    items = [(d['label'], r) for r, d in zip(rs, pltdata)]
    legend = Legend(items=items, location=(10, -30))
    legend.label_text_font_size = '8pt'
    
    p2.add_layout(legend, 'right')  
    
    show(gridplot(p1, p2, ncols=2, plot_height=500))
    

def script_plt_cooling():
    pltdata = pltdata_cooling_1(
        ds_cool=DS_COOL,
        ds_cool_crd=DS_COOL_CRD,
        ds_cool_wgt=DS_COOL_WGT)
    plt_cooling_bokeh(pltdata=pltdata)
    print('''FIGURE. Cooling rate profiles.''')

In [9]:
def fmt_flux(ds_in, ilevels=None):
    ds = ds_in.copy(deep=True)
    
    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    if ilevels:
        ds = ds.isel(pressure=ilevels)
        
    if 'igg' in ds.dims:
        ds = ds.sel(igg=[1])
        ds = ds.squeeze('igg').drop('igg')
    
    if 'g' in ds.dims:
        ds = ds.sum('g')
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band').drop('band')
        except ValueError:
            ds = ds.sum('band')
                
    df = ds.to_dataframe()
    df = df.set_index('level', append=True)               
    return df


def tbdata_flux(ds_flux=None, ds_flux_crd=None, ds_flux_wgt=None):
    ilevels = [0, 23, -1]
    
    benchmark = {'label': 'CRD',
                 'df': fmt_flux(ds_flux_crd, ilevels=ilevels)}
    
    tbs = [{'label': 'WGT igg=10 wgt_flux=1',
            'df': fmt_flux(ds_flux_wgt, ilevels=ilevels)},
           {'label': 'CLIRAD',
            'df': fmt_flux(ds_flux, ilevels=ilevels)}]
    
    data = {'benchmark': benchmark, 'others': tbs}
    return data


def tb_flux(tbdata=None):
    benchmark = tbdata['benchmark']
    others = tbdata['others']
    
    def show_tb(d=None):
        display.display(d['df'])
        print('Table.', 'Fluxes.', d['label'])
        
    def show_tbdiff(d0=None, d=None):
        df = d['df']
        df.index = d0['df'].index
        display.display(df - d0['df'])
        print('Table.', 'Fluxes.', 
              '({}) - ({})'.format(d['label'], d0['label']))
        
    show_tb(benchmark)
    for d in others:
        show_tb(d)
        
    for d in others:
        show_tbdiff(d=d, d0=benchmark)
        
        
def script_tb_flux():
    data = tbdata_flux(ds_flux=DS_FLUX,
                       ds_flux_crd=DS_FLUX_CRD,
                       ds_flux_wgt=DS_FLUX_WGT)
    tb_flux(tbdata=data)

In [10]:
def band_map():
    '''
    Maps spectral bands in lblnew to spectral bands in clirad.
    '''
    lblnew2clirad = {'1': '1',
            '2': '2',
            '3a': '3',
            '3b': '4',
            '3c': '5',
            '4': '6', 
            '5': '7',
            '6': '8',
            '7': '9', 
            '8': '10',
            '9': '11'}
    
    clirad2lblnew = {clirad: lblnew 
                     for lblnew, clirad in lblnew2clirad.items()}
    return clirad2lblnew


def gasbands():
    bmap = band_map()
    gases = list(PARAM['molecule'].keys())
    
    bands = [bmap[str(b)] for b in PARAM['band']]
    return list(itertools.product(gases, bands))


def print_bestfit_params():
    print('Best-fit values for each (gas, band)')
    print('------------------------------------')
    for gas, band in gasbands():
        print('o {} band{}'.format(gas, band))
        param = bestfits.kdist_params(molecule=gas, band=band)
        for name, value in sorted(param.items()):
            print(2 * ' ' + '{} = {}'.format(name, value))

        
def print_input_param():        
    for name, value in PARAM.items():
        print('{} = {}'.format(name, value))
        
        
def print_lblnew_param():
    for name, value in sorted(PARAM_LBLNEW.items()):
        print('{} = {}'.format(name, value))


In [11]:
'''
Cooling rate profile by layer
'''
def script_coolr_bylayer():
    '''
    Displays cooling rate of each layer
    '''
    keys = ['CRD', 'CLIRAD `coolr_bands`']
    dss = [DS_COOL_CRD, DS_COOL]
    das = [fmt_cool(ds) for ds in dss]
    
    dfs = [da.to_dataframe() for da in das]
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'coolr_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))



In [12]:
'''
Fluxes by layer
'''
def script_flux_bylayer():
    '''
    Display fluxes of each layer
    '''
    keys = ['CRD', 'CLIRAD']
    dss = [DS_FLUX_CRD, DS_FLUX]
    dfs = [fmt_flux(ds) for ds in dss]
    
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'fluxes_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))


In [13]:
def show_html(s):
    display.display(display.HTML(s))

    
def show_markdown(s):
    display.display(display.Markdown(s))

    
def script():
    
    title = '### CLIRAD-LW'
    s_input = 'Input Parameters'
    s_lblnew_param = 'LBLNEW case compaired against'
    s_bestfits = 'Best-fit Parameters'
    s_plt_cooling = 'Cooling Rate Profiles'
    s_tb_flux = 'Flux Comparison'
    s_coolr_bylayer = 'Cooling Rate by Layer'
    s_flux_bylayer = 'Fluxes by Layer'
    
    # title
    show_markdown('### CLIRAD-LW')
    
    # TOC
    show_markdown('### Table of Contents')
    show_html(climahtml.getHTML_hrefanchor(s_input))
    show_html(climahtml.getHTML_hrefanchor(s_lblnew_param))
    show_html(climahtml.getHTML_hrefanchor(s_plt_cooling))
    show_html(climahtml.getHTML_hrefanchor(s_tb_flux))
    show_html(climahtml.getHTML_hrefanchor(s_bestfits))
    show_html(climahtml.getHTML_hrefanchor(s_coolr_bylayer))
    show_html(climahtml.getHTML_hrefanchor(s_flux_bylayer))
    
    # Input parameters
    show_html(climahtml.getHTML_idanchor(s_input))
    show_markdown(climahtml.getMarkdown_sectitle(s_input))
    print_input_param()
    
    # LBLNEW parameters 
    show_html(climahtml.getHTML_idanchor(s_lblnew_param))
    show_markdown(climahtml.getMarkdown_sectitle(s_lblnew_param))
    print_lblnew_param()
    
    # Cooling rate profiles
    show_html(climahtml.getHTML_idanchor(s_plt_cooling))
    show_markdown(climahtml.getMarkdown_sectitle(s_plt_cooling))
    script_plt_cooling()
    
    # Flux comparison
    show_html(climahtml.getHTML_idanchor(s_tb_flux))
    show_markdown(climahtml.getMarkdown_sectitle(s_tb_flux))
    script_tb_flux()
    
    # Best-fit values
    show_html(climahtml.getHTML_idanchor(s_bestfits))
    show_markdown(climahtml.getMarkdown_sectitle(s_bestfits))
    print_bestfit_params()
    
    # Cooling rate by layer
    show_html(climahtml.getHTML_idanchor(s_coolr_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_coolr_bylayer))
    script_coolr_bylayer()
    
    # Fluxes by layer
    show_html(climahtml.getHTML_idanchor(s_flux_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_flux_bylayer))
    script_flux_bylayer()
   
    

In [14]:
script()

### CLIRAD-LW

### Table of Contents

# Input Parameters

atmpro = mls
band = [1]
commitnumber = de5d43e
molecule = {'h2o': 'atmpro'}
tsfc = 294


# LBLNEW case compaired against

...

Best-fit values for each (gas, band)
------------------------------------
o h2o band1
  atmpro = mls
  band = 1
  commitnumber = a06b618
  conc = None
  dv = 0.001
  klin = 0
  molecule = h2o
  ng_adju = [0, 0, 0]
  ng_refs = [3, 3, 6]
  nv = 1000
  option_compute_btable = 0
  option_compute_ktable = 1
  option_wgt_flux = 1
  option_wgt_k = 1
  ref_pts = [(1, 250), (30, 250), (300, 250)]
  tsfc = 294
  vmax = 340
  vmin = 20
  w_diffuse = [(1.66, 1.66, 1.66), (1.66, 1.66, 1.66), (1.66, 1.66, 1.5, 1.5, 1.5, 1.5)]
  wgt = [(0.2, 0.2, 0.2), (0.2, 0.2, 0.2), (0.2, 0.2, 0.65, 0.65, 0.65, 0.65)]


# Cooling Rate by Layer

CRD           CLIRAD `coolr_bands`          
            layer    coolrg                layer    coolrg
pressure                                                  
0.000312      1.0  6.538142                  NaN       NaN
0.000312      NaN       NaN                  1.0  3.930345
0.000750      2.0  5.055378                  2.0  3.665455
0.001052      3.0  4.528686                  3.0  3.532528
0.001476      4.0  3.991609                  4.0  3.345939
0.002070      5.0  3.463416                  5.0  3.094162
0.002904      6.0  2.964688                  6.0  2.774866
0.004074      7.0  2.507538                  7.0  2.397780
0.005714      8.0  2.095884                  8.0  1.989776
0.008015      9.0  1.729136                  9.0  1.594391
0.011243     10.0  1.406249                 10.0  1.260502
0.015771     11.0  1.126271                 11.0  1.016425
0.022122     12.0  0.886746                 12.0  0.846264
0.031031     13.0  0.682595                 13.0  0.706796
0.043528     14.0  0.498515                 14.0  0.553505
0.061057     15.0  0.368263                 15.0  0.412932
0.085645     16.0  0.349431                 16.0  0.356630
0.120136     17.0  0.375653                 17.0  0.349521
0.168516     18.0  0.385030                 18.0  0.349892
0.236378     19.0  0.396302                 19.0  0.371380
0.331549     20.0  0.418323                 20.0  0.406092
0.465100     21.0  0.457010                 21.0  0.445696
0.652400     22.0  0.524987                 22.0  0.510027
0.915100     23.0  0.606909                 23.0  0.610823
1.283650     24.0  0.618638                 24.0  0.663257
1.800600     25.0  0.570939                 25.0  0.642821
2.525700     26.0  0.529225                 26.0  0.588970
3.542800     27.0  0.494019                 27.0  0.534227
4.969550     28.0  0.461344                 28.0  0.467432
6.970850     29.0  0.432716                 29.0  0.418874
9.778100     30.0  0.406825                 30.0  0.398613
13.715850    31.0  0.378515                 31.0  0.374704
19.239350    32.0  0.345810                 32.0  0.338653
26.987250    33.0  0.318185                 33.0  0.319207
37.855300    34.0  0.298341                 34.0  0.301203
53.100050    35.0  0.275279                 35.0  0.272005
73.887500    36.0  0.247625                 36.0  0.238734
97.662500    37.0  0.228765                 37.0  0.210431
121.437500   38.0  0.218934                 38.0  0.196074
145.212500   39.0  0.201939                 39.0  0.181074
168.987500   40.0  0.192083                 40.0  0.175044
192.762500   41.0  0.298105                 41.0  0.279785
216.537500   42.0  0.719739                 42.0  0.692359
240.312500   43.0  1.142834                 43.0  1.121270
264.087500   44.0  1.371455                 44.0  1.425955
287.862500   45.0  1.367191                 45.0  1.383613
311.637500   46.0  1.090029                 46.0  1.105924
335.412500   47.0  0.874029                 47.0  0.930630
359.187500   48.0  0.728956                 48.0  0.755233
382.962500   49.0  0.610124                 49.0  0.605180
406.737500   50.0  0.501075                 50.0  0.502904
430.512500   51.0  0.397690                 51.0  0.416488
454.287500   52.0  0.297351                 52.0  0.312568
478.062500   53.0  0.212697                 53.0  0.209276
501.837500   54.0  0.146253                 54.0  0.125742
525.612500   55.0  0.096576                 55.0  0.069743
549.387500   56.0  0.061861                 56.0  0.037727
573.162500   57.0  0.039594                 57.0  0.022546
596.937500   58.0  0.026239                 58.0  0.015721
620.712500   59.0  0.016982                 59.0  0.011279
644.487500   60.0  0.011297                 60.0  0.008440
668.262500   61.0  0.008214                 61.0  0.006643
692.037500   62.0  0.006487                 62.0  0.005549
715.812500   63.0  0.005621                 63.0  0.004994
739.587500   64.0  0.005374                 64.0  0.004994
76

# Fluxes by Layer

CRD                           CLIRAD                \
                         flug       fldg      fnetg       flug          fldg   
pressure     level                                                             
0.000000e+00 1     -34.263992   0.000000 -34.263992        NaN           NaN   
1.000000e-08 1            NaN        NaN        NaN -34.073650  5.670603e-07   
6.244000e-04 2     -34.264010   0.000501 -34.263509 -34.073667  3.078297e-04   
8.759000e-04 3     -34.264017   0.000659 -34.263358 -34.073673  4.236623e-04   
1.228600e-03 4     -34.264026   0.000858 -34.263169 -34.073682  5.799128e-04   
1.723400e-03 5     -34.264038   0.001103 -34.262935 -34.073693  7.872701e-04   
2.417400e-03 6     -34.264054   0.001403 -34.262650 -34.073708  1.056221e-03   
3.390900e-03 7     -34.264073   0.001765 -34.262308 -34.073727  1.395145e-03   
4.756500e-03 8     -34.264099   0.002196 -34.261903 -34.073751  1.807662e-03   
6.672000e-03 9     -34.264132   0.002704 -34.261427 -34.073784  2.291372e-03   
9.358900e-03 10    -34.264175   0.003298 -34.260877 -34.073826  2.841278e-03   
1.312780e-02 11    -34.264232   0.003983 -34.260249 -34.073883  3.460773e-03   
1.841450e-02 12    -34.264311   0.004767 -34.259544 -34.073960  4.175110e-03   
2.583020e-02 13    -34.264421   0.005656 -34.258765 -34.074070  5.028482e-03   
3.623230e-02 14    -34.264585   0.006661 -34.257924 -34.074233  6.062427e-03   
5.082340e-02 15    -34.264863   0.007801 -34.257062 -34.074506  7.292219e-03   
7.129060e-02 16    -34.265347   0.009178 -34.256169 -34.074973  8.759783e-03   
1.000000e-01 17    -34.266043   0.011062 -34.254981 -34.075630  1.063011e-02   
1.402710e-01 18    -34.266899   0.013710 -34.253189 -34.076425  1.309242e-02   
1.967600e-01 19    -34.267931   0.017318 -34.250612 -34.077362  1.637171e-02   
2.759970e-01 20    -34.269147   0.022254 -34.246893 -34.078436  2.093143e-02   
3.871000e-01 21    -34.270515   0.029128 -34.241387 -34.079583  2.742401e-02   
5.431000e-01 22    -34.271879   0.038938 -34.232942 -34.080604  3.668159e-02   
7.617000e-01 23    -34.272541   0.053193 -34.219347 -34.080763  5.004861e-02   
1.068500e+00 24    -34.270850   0.073560 -34.197291 -34.078435  6.992211e-02   
1.498800e+00 25    -34.266117   0.100360 -34.165757 -34.072948  9.824645e-02   
2.102400e+00 26    -34.259153   0.134219 -34.124935 -34.065069  1.363340e-01   
2.949000e+00 27    -34.250007   0.178145 -34.071861 -34.054887  1.852237e-01   
4.136600e+00 28    -34.238365   0.236003 -34.002363 -34.042075  2.475747e-01   
5.802500e+00 29    -34.223948   0.312626 -33.911322 -34.026485  3.242354e-01   
8.139200e+00 30    -34.206152   0.414606 -33.791547 -34.007416  4.211226e-01   
1.141700e+01 31    -34.184038   0.550453 -33.633585 -33.984082  5.525779e-01   
1.601470e+01 32    -34.157724   0.730289 -33.427434 -33.956630  7.292219e-01   
2.246400e+01 33    -34.130450   0.967203 -33.163247 -33.928367  9.597048e-01   
3.151050e+01 34    -34.104899   1.282626 -32.822273 -33.902042  1.275486e+00   
4.420010e+01 35    -34.080035   1.706221 -32.373815 -33.877008  1.703259e+00   
6.200000e+01 36    -34.062659   2.269277 -31.793382 -33.860622  2.260462e+00   
8.577500e+01 37    -34.075578   2.979587 -31.095991 -33.876580  2.948841e+00   
1.095500e+02 38    -34.117337   3.665620 -30.451717 -33.922123  3.587087e+00   
1.333250e+02 39    -34.175205   4.340075 -29.835130 -33.984078  4.201306e+00   
1.571000e+02 40    -34.261221   4.994817 -29.266404 -34.074606  4.801848e+00   
1.808750e+02 41    -34.394933   5.669496 -28.725437 -34.212543  5.432816e+00   
2.046500e+02 42    -34.597482   6.711604 -27.885878 -34.420554  6.428873e+00   
2.284250e+02 43    -34.924059   9.065195 -25.858864 -34.755868  8.714292e+00   
2.522000e+02 44    -35.366352  12.726073 -22.640279 -35.212917  1.232952e+01   
2.759750e+02 45    -35.915643  17.137817 -18.777826 -35.778450  1.691141e+01   
2.997500e+02 46    -36.564682  21.637301 -14.927381 -36.448059  2.147812e+01   
3.235250e+02 47    -37.247

In [15]:
display.HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')